In [1]:
import pandas as pd
import numpy as np
import numpy as np

First, I check the structure of the "xls" file, and I could determine which sheets I can to use for the analysis.

In [11]:
xls = pd.ExcelFile("FoodCompAfricaWest.xlsx")
food = pd.read_excel(xls, 2)

I will eliminate the rows with a number of N/A values greater than 44 also I will eliminate the first two rows.

In [20]:
condition = food.isna().sum(axis=1) > 44
food.drop(food[condition].index, inplace=True)
food.drop(index=[0,1], inplace=True)

,Code,Food name in English,Food name in French,Scientific name,BiblioID/Source,Edible portion coefficient 1 (from as purchased to as described),Energy\n(kJ),Energy\n(kcal),Water\n(g),"Protein, total\n(g)",...,"Folate, dietary folate equivalents\n(mcg)",Vitamin B12\n(mcg),Vitamin C\n(mg),Cholesterol\n(mg),"Fatty acids, total saturated\n(g)","Fatty acids, total monounsaturated \n(g)","Fatty acids, total polyunsaturated\n(g)",Linoleic acid \n(g),Alpha-linolenic acid\n(g),"Phytate, total or [phytate, determined by direct precipitation] or [phytate, determined by indirect precipitation]\n(mg)"
2,Cereals and their products/Céréales et produit...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01_172,Baling béinré (northern Burkina Faso)*: sorghu...,Baling béinré (nord du Burkina Faso)*: bouilli...,NaN,calc. from recipe / de recette,NaN,329,78,80.1,2.5,...,[4],0.12,2,1,[0.06],[0.05],[0.05],[0.04],[0.01],[23]
4,01_173,Baling ni zièm béinré (Burkina Faso)*: porridg...,Baling ni zièm béinré (Burkina Faso)*: bouilli...,NaN,calc. from recipe / de recette,NaN,277,65,83.3,0.9,...,[2],0.00,0,0,0.03,0.04,0.04,0.04,0.00,21
5,01_168,Beenkida (Burkina Faso)*: maize porridge with ...,Beenkida (Burkina Faso)*: bouillie de maïs ave...,NaN,calc. from recipe / de recette,NaN,426,100,74.6,2.1,...,1,0.00,0,0,0.05,0.12,0.13,0.12,0.00,41
6,01_188,"Biscuit, sweet, plain, unfortified","Biscuit, sucré, nature, non enrichi",NaN,"AU14(02C20287; 02C20364; 02C20286; 02C20314), ...",1.00,2010,479,4.6,6.2,...,12,0.06,1,30,10.30,7.65,2.38,2.03,0.29,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1052,14_024,Téi né maane zéindo (Burkina Faso)*: oil palm ...,Téi né maane zéindo (Burkina Faso)*: noix de p...,NaN,calc. from recipe / de recette,NaN,399,96,83.5,2.7,...,[9],0.95,3,6,2.77,2.83,0.80,0.09,0.01,[1]
1053,14_023,Téi zéindo (Burkina Faso)*: simple oil palm fr...,Téi zéindo (Burkina Faso)*: sauce simple de no...,NaN,calc. from recipe / de recette,NaN,823,199,71.5,1.7,...,[1],0.42,4,3,6.47,6.85,1.69,0.18,0.01,[0]
1054,14_020,Toeg koing zéédo (Burkina Faso)*: dried baobab...,Toeg koing zéédo (Burkina Faso)*: sauce de feu...,NaN,calc. from recipe / de recette,NaN,181,44,89.1,2.8,...,[11],[0.79],5,5,0.50,0.69,1.09,0.95,0.04,[1]
1055,14_022,Toeg maas zéédo (Burkina Faso)*: fresh baobab ...,Toeg maas zéédo (Burkina Faso)*: sauce de feui...,NaN,calc. from recipe / de recette,NaN,251,60,86.7,3.1,...,[14],[0.92],9,6,0.74,1.20,1.87,1.71,0.04,[7]
